# Customer Churn Prediction with MLflow Experiment Tracking

This notebook demonstrates end-to-end machine learning experiment tracking using MLflow for customer churn prediction.

## Features:
- Comprehensive experiment tracking
- Model comparison and selection
- Automated model registry
- Reproducible experiments
- Detailed performance analysis

## 1. Setup MLflow Environment

In [ ]:
# Install MLflow and dependencies
import subprocess
import sys

def install_package(package):
    """Install a package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"Successfully installed {package}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package}")

# Install required packages
packages = ['mlflow', 'xgboost', 'scikit-learn', 'pandas', 'numpy', 'matplotlib', 'seaborn']
for package in packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"✓ {package} is available")
    except ImportError:
        print(f"Installing {package}...")
        install_package(package)

In [ ]:
# Import libraries
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import os
import joblib

# Suppress warnings
warnings.filterwarnings('ignore')

# Configuration parameters
USE_AWS_MLFLOW = False  # Set to True to use AWS SageMaker MLflow
MLFLOW_TRACKING_ARN = "arn:aws:sagemaker:us-east-1:{123456789}:mlflow-tracking-server/your-server-name"  # Replace with your actual MLflow ARN

# Configure MLflow tracking
if USE_AWS_MLFLOW:
    # AWS SageMaker MLflow tracking server
    mlflow.set_tracking_uri(MLFLOW_TRACKING_ARN)
    print(f"📊 Using AWS SageMaker MLflow tracking")
    print(f"🔗 Tracking ARN: {MLFLOW_TRACKING_ARN}")
else:
    # Local MLflow tracking
    mlflow.set_tracking_uri("./mlruns")
    print(f"🏠 Using local MLflow tracking")
    print(f"📁 Tracking URI: {mlflow.get_tracking_uri()}")

# Create experiment
experiment_name = f"customer_churn_automl_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
try:
    mlflow.set_experiment(experiment_name)
    print(f"✅ Experiment created: {experiment_name}")
except Exception as e:
    print(f"⚠️ Experiment creation issue: {e}")
    print("Continuing with default experiment...")

print(f"🔢 MLflow Version: {mlflow.__version__}")
print(f"🎯 Experiment Focus: AutoML model metrics tracking only")

## 2. Data Loading and Initial Exploration

In [ ]:
# Load and prepare data for AutoML
print("🚀 Loading and preparing data for AutoML...")

# Load dataset
data_path = "./data/customer_churn.csv"
df = pd.read_csv(data_path)

print(f"📊 Dataset shape: {df.shape}")
print(f"🎯 Target column: {'Churn' if 'Churn' in df.columns else 'Not found'}")

# Basic data info (no MLflow logging for data exploration)
churn_rate = (df['Churn'].sum() / len(df)) * 100
print(f"📈 Churn rate: {churn_rate:.2f}%")
print(f"🔢 Total samples: {len(df)}")
print(f"📋 Features: {len(df.columns) - 1}")  # Excluding target

# Quick data quality check
missing_values = df.isnull().sum().sum()
if missing_values > 0:
    print(f"⚠️ Missing values found: {missing_values}")
else:
    print("✅ No missing values detected")

print("📦 Data ready for AutoML processing...")

## 3. Data Preprocessing Pipeline with Tracking

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

def prepare_data_for_automl(df):
    """Streamlined data preparation for AutoML"""
    print("🔄 Preparing data for AutoML...")
    
    # Make a copy
    processed_df = df.copy()
    
    # Handle missing values
    if processed_df.isnull().sum().sum() > 0:
        processed_df = processed_df.dropna()
        print(f"🧹 Dropped rows with missing values")
    
    # Remove CustomerID if present
    if 'CustomerID' in processed_df.columns:
        processed_df = processed_df.drop('CustomerID', axis=1)
    
    # Define feature types
    numerical_cols = processed_df.select_dtypes(include=[np.number]).columns.tolist()
    if 'Churn' in numerical_cols:
        numerical_cols.remove('Churn')
    
    categorical_cols = processed_df.select_dtypes(include=['object']).columns.tolist()
    
    print(f"📊 Numerical features: {len(numerical_cols)}")
    print(f"📊 Categorical features: {len(categorical_cols)}")
    
    # Separate features and target
    X = processed_df.drop('Churn', axis=1)
    y = processed_df['Churn']
    
    # Create simple preprocessing pipeline for AutoML
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols)
        ]
    )
    
    # Fit and transform
    X_processed = preprocessor.fit_transform(X)
    
    # Get feature names for interpretability
    try:
        cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
        all_feature_names = numerical_cols + list(cat_feature_names)
    except:
        all_feature_names = [f"feature_{i}" for i in range(X_processed.shape[1])]
    
    print(f"✅ Data preprocessing completed")
    print(f"📈 Final feature count: {X_processed.shape[1]}")
    
    return X_processed, y, preprocessor, all_feature_names

# Prepare data
X_processed, y, preprocessor, feature_names = prepare_data_for_automl(df)

# Split data for AutoML
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y
)

print(f"🏋️ Training set: {X_train.shape}")
print(f"🧪 Test set: {X_test.shape}")
print("🎯 Ready for AutoML model training!")

## 4. Model Training with MLflow Experiments

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
import time

# Try to import additional AutoML libraries
try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    xgb_available = False
    print("⚠️ XGBoost not available")

try:
    from lightgbm import LGBMClassifier
    lgb_available = True
except ImportError:
    lgb_available = False
    print("⚠️ LightGBM not available")

print("🤖 Starting AutoML Model Training with MLflow Tracking")
print("=" * 60)

# Define AutoML model suite
automl_models = {
    'Random_Forest': {
        'model': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
        'params': {'n_estimators': 100, 'max_depth': None, 'random_state': 42}
    },
    'Gradient_Boosting': {
        'model': GradientBoostingClassifier(n_estimators=100, random_state=42),
        'params': {'n_estimators': 100, 'learning_rate': 0.1, 'random_state': 42}
    },
    'Logistic_Regression': {
        'model': LogisticRegression(random_state=42, max_iter=1000),
        'params': {'C': 1.0, 'max_iter': 1000, 'random_state': 42}
    },
    'SVM_RBF': {
        'model': SVC(kernel='rbf', probability=True, random_state=42),
        'params': {'kernel': 'rbf', 'C': 1.0, 'random_state': 42}
    }
}

# Add optional models if available
if xgb_available:
    automl_models['XGBoost'] = {
        'model': XGBClassifier(random_state=42, eval_metric='logloss'),
        'params': {'n_estimators': 100, 'max_depth': 6, 'random_state': 42}
    }

if lgb_available:
    automl_models['LightGBM'] = {
        'model': LGBMClassifier(random_state=42, verbose=-1),
        'params': {'n_estimators': 100, 'num_leaves': 31, 'random_state': 42}
    }

print(f"🎯 AutoML will evaluate {len(automl_models)} models")

# AutoML Training Loop with MLflow
model_results = {}

for model_name, config in automl_models.items():
    with mlflow.start_run(run_name=f"automl_{model_name}") as run:
        print(f"\n🔥 Training: {model_name}")
        
        # Log only essential model parameters
        mlflow.log_param("model_algorithm", model_name)
        mlflow.log_param("automl_mode", True)
        
        model = config['model']
        start_time = time.time()
        
        # Cross-validation for robust evaluation
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
        
        # Train final model
        model.fit(X_train, y_train)
        training_time = time.time() - start_time
        
        # Generate predictions
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else y_pred
        
        # Calculate comprehensive metrics
        metrics = {
            'test_accuracy': accuracy_score(y_test, y_pred),
            'test_roc_auc': roc_auc_score(y_test, y_pred_proba),
            'test_precision': precision_score(y_test, y_pred),
            'test_recall': recall_score(y_test, y_pred),
            'test_f1_score': f1_score(y_test, y_pred),
            'cv_roc_auc_mean': cv_scores.mean(),
            'cv_roc_auc_std': cv_scores.std(),
            'training_time_seconds': training_time
        }
        
        # Log metrics to MLflow
        for metric_name, metric_value in metrics.items():
            mlflow.log_metric(metric_name, metric_value)
        
        # Log model to MLflow
        if model_name == 'XGBoost' and xgb_available:
            mlflow.xgboost.log_model(model, "model")
        elif model_name == 'LightGBM' and lgb_available:
            mlflow.lightgbm.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")
        
        # Store results for comparison
        model_results[model_name] = {
            'model': model,
            'metrics': metrics,
            'run_id': run.info.run_id
        }
        
        # Print summary
        print(f"  ✅ ROC-AUC: {metrics['test_roc_auc']:.4f}")
        print(f"  📊 Accuracy: {metrics['test_accuracy']:.4f}")
        print(f"  ⏱️ Time: {training_time:.2f}s")

print(f"\n🎉 AutoML Training Complete! Evaluated {len(model_results)} models")
print("📊 All model metrics logged to MLflow")

## 5. Model Comparison and Selection

In [ ]:
# AutoML Model Comparison and Selection
with mlflow.start_run(run_name="automl_comparison") as run:
    
    print("🏆 AutoML Model Comparison Results")
    print("=" * 50)
    
    # Create comparison DataFrame
    comparison_data = []
    for model_name, results in model_results.items():
        metrics = results['metrics']
        comparison_data.append({
            'Model': model_name,
            'ROC-AUC': metrics['test_roc_auc'],
            'Accuracy': metrics['test_accuracy'],
            'Precision': metrics['test_precision'],
            'Recall': metrics['test_recall'],
            'F1-Score': metrics['test_f1_score'],
            'CV_ROC_AUC': metrics['cv_roc_auc_mean'],
            'Training_Time': metrics['training_time_seconds']
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df = comparison_df.sort_values('ROC-AUC', ascending=False)
    
    print("\n📊 Model Performance Ranking:")
    print(comparison_df.round(4))
    
    # Select best model
    best_model_name = comparison_df.iloc[0]['Model']
    best_model_metrics = model_results[best_model_name]['metrics']
    best_run_id = model_results[best_model_name]['run_id']
    
    # Log comparison metrics to MLflow
    mlflow.log_param("automl_models_compared", len(model_results))
    mlflow.log_param("best_model_algorithm", best_model_name)
    mlflow.log_metric("best_roc_auc", best_model_metrics['test_roc_auc'])
    mlflow.log_metric("best_accuracy", best_model_metrics['test_accuracy'])
    mlflow.log_metric("models_evaluated", len(model_results))
    
    # Create simple performance visualization
    plt.figure(figsize=(12, 6))
    
    # ROC-AUC comparison
    plt.subplot(1, 2, 1)
    bars = plt.bar(comparison_df['Model'], comparison_df['ROC-AUC'], color='skyblue', alpha=0.7)
    plt.title('AutoML Model Performance (ROC-AUC)', fontsize=14, fontweight='bold')
    plt.ylabel('ROC-AUC Score')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    
    # Highlight best model
    bars[0].set_color('gold')
    bars[0].set_alpha(1.0)
    
    # Training time comparison
    plt.subplot(1, 2, 2)
    plt.bar(comparison_df['Model'], comparison_df['Training_Time'], color='lightcoral', alpha=0.7)
    plt.title('Training Time Comparison', fontsize=14, fontweight='bold')
    plt.ylabel('Training Time (seconds)')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    # Save and log visualization
    plot_path = "automl_comparison.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    mlflow.log_artifact(plot_path)
    plt.show()
    
    print(f"\n🥇 Best AutoML Model: {best_model_name}")
    print(f"   🎯 ROC-AUC: {best_model_metrics['test_roc_auc']:.4f}")
    print(f"   📊 Accuracy: {best_model_metrics['test_accuracy']:.4f}")
    print(f"   🎲 Precision: {best_model_metrics['test_precision']:.4f}")
    print(f"   📈 Recall: {best_model_metrics['test_recall']:.4f}")
    print(f"   ⚖️ F1-Score: {best_model_metrics['test_f1_score']:.4f}")
    print(f"   ⏱️ Training Time: {best_model_metrics['training_time_seconds']:.2f}s")
    
    print(f"\n📋 AutoML Summary:")
    print(f"   • Models Evaluated: {len(model_results)}")
    print(f"   • Best Performance: {best_model_metrics['test_roc_auc']:.4f} ROC-AUC")
    print(f"   • Performance Range: {comparison_df['ROC-AUC'].min():.4f} - {comparison_df['ROC-AUC'].max():.4f}")
    
    # Store best model info for next steps
    best_model_info = {
        'name': best_model_name,
        'run_id': best_run_id,
        'metrics': best_model_metrics,
        'model_uri': f"runs:/{best_run_id}/model"
    }